<a href="https://colab.research.google.com/github/larissabtn/desafio-2-dio/blob/main/Desafio_autoria_LARISSABTN_de_SantanderBootcamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [1]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [2]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[3334, 3335, 3368]


In [3]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 3334,
    "name": "Marcos",
    "account": {
      "id": 3538,
      "number": "00003-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 2000.0
    },
    "card": {
      "id": 3228,
      "number": "**** **** **** 1236",
      "limit": 2000.0
    },
    "features": [
      {
        "id": 1030,
        "icon": "string",
        "description": "string"
      }
    ],
    "news": [
      {
        "id": 6773,
        "icon": "string",
        "description": "string"
      },
      {
        "id": 7176,
        "icon": "https://raw.githubusercontent.com/digitalinnovationone/santander-dev-week-2023-api/665804c8231b32923b03b30ab832ccee33cf2cc3/docs/icons/account.svg",
        "description": "Maximize seu potencial financeiro e conquiste seus objetivos atrav\u00e9s dos investimentos. Seu futuro come\u00e7a agora! \ud83d\udcb0\ud83d\udcc8 #invista"
      },
      {
        "id": 7177,
        "icon": "https://raw.githubusercontent.com/digitalinnovationone/santan

## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [5]:
def get_motivational_messages(user):
  df = pd.read_csv('SDW2023.csv')
  messages = df['Message'].tolist()

  return messages


for index, user in enumerate(users):
  messages = get_motivational_messages(user)
  message = messages[index]

  print(message)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": message
  })

Investimentos são legais
Investimentos são maneiros
Investimentos são cool


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [6]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Marcos updated? True!
User João updated? True!
User Alfonso updated? True!
